In [1]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

from config import OPENAI_API_KEY
from tools import create_rag_tool, create_google_search_tool, create_website_opener_tool

In [2]:
tools = [
    create_rag_tool('simple_graduation_rules.txt', 'utf-8', 150, 50, 'simple_graduation_rules_retriever', '서울대학교 산업공학과 졸업을 위한 졸업 요건에 대한 간단한 질문에 답할 때 사용'),
    create_rag_tool('detailed_graduation_rules.pdf', 'utf-8', 300, 100, 'detailed_graduation_rules_retriever', '서울대학교 산업공학과 졸업을 위한 졸업 요건에 대한 복잡한 질문에 답할 때 사용'),
    create_google_search_tool(),
    create_website_opener_tool()
]

llm = ChatOpenAI(
    model='gpt-4-turbo',
    api_key=OPENAI_API_KEY,
    temperature=0
)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [3]:
response1 = agent_executor.invoke({
    "input" : "서울대학교 산업공학과를 졸업하기 위해 필요한 학점은 얼마나 필요해? 전공, 교양 등 각기 영역별로 필요한 학점을 알려줘."
})



> Entering new AgentExecutor chain...
To answer the question about the specific credit requirements for graduation from the Industrial Engineering department at Seoul National University, including details for major and general education credits, I should use the detailed graduation rules retriever tool. This tool is designed to provide complex and detailed information about graduation requirements.

Action: detailed_graduation_rules_retriever
Action Input: "서울대학교 산업공학과 졸업 학점 요건"가. 교과목 이수규정
⑴ 학사과정 졸업(수료) 이수학점
기준년도 서울대학교 교과과정 산업공학과 내규 비고졸업학점 교양학점 주전공학점 전필학점1 학과 전선 학점
2008 130 54(55) 62 34 12학점 이상
2009 130 55(56) 62 34 12학점 이상 대학영어 이수학점
상향 조정
2010 130 56 62 34 12학점 이상 컴퓨터의 원리(기초)
폐지
2011–2012 130 53 62 37 12학점 이상 공학소양 1 · 2 통합
130 47 62 34 15학점 이상

가. 교과목 이수규정
⑴ 학사과정 졸업(수료) 이수학점
기준년도 서울대학교 교과과정 산업공학과 내규 비고졸업학점 교양학점 주전공학점 전필학점1 학과 전선 학점
2008 130 54(55) 62 34 12학점 이상
2009 130 55(56) 62 34 12학점 이상 대학영어 이수학점
상향 조정
2010 130 56 62 34 12학점 이상 컴퓨터의 원리(기초)
폐지
2011–2012 130 53 62 37 12학점 이상 공학소양

In [11]:
response2 = agent_executor.invoke({
    "input" : "서울대학교 산업공학과의 최신 졸업 이수 규정을 검색해줘."
})



> Entering new AgentExecutor chain...
To find the latest graduation requirements for the Department of Industrial Engineering at Seoul National University, I should use a tool that can access the most current and specific information.

Action: google_web_search
Action Input: "Seoul National University Industrial Engineering latest graduation requirements"SNU has one Graduate School with 106 programs across five fields of study. Interdisciplinary programs are operated by more than two different departments. Department / College, Graduate School · Civil and Environmental Engineering · Architecture and Architectural Engineering · Mechanical Engineering · Industrial ... SNU's 15 Colleges offer various undergraduate degree programs. Those who want to study in 'interdisciplinary programs for undergraduate's or 'combined minor' ... Graduation Requirementsopenopen. Undergraduate · graduate · Researchopen ... COPYRIGHT c 2025 Department of Mechanical Engineering, Seoul National University. Oc

In [12]:
response3 = agent_executor.invoke({
    "input" : "현재 내 서울대학교 학업 이수 현황을 알고 싶어. 관련 웹페이지를 연결해줘."
})



> Entering new AgentExecutor chain...
The user wants to access their academic progress status at Seoul National University. The best way to provide this information is by opening the relevant webpage where they can directly check their status.

Action: website_opener
Action Input: 서울대학교 학업 이수 현황성공적으로 '서울대학교 학업 이수 현황' [https://snugenie.snu.ac.kr/mypage/stdCmplCond.do]을 브라우저에서 열었음.I now know the final answer
Final Answer: 서울대학교 학업 이수 현황 페이지를 성공적으로 열었습니다. 해당 페이지에서 직접 자신의 학업 이수 현황을 확인할 수 있습니다.

> Finished chain.


In [4]:
print(response1['output'])

서울대학교 산업공학과 졸업을 위해서는 다음과 같은 학점이 필요합니다:
- 총 졸업학점: 130학점
- 교양학점: 최소 47학점에서 56학점 사이 (년도에 따라 다름)
- 주전공학점 (전필학점 + 학과 전선 학점): 62학점
  - 전필학점: 34학점에서 37학점 사이 (년도에 따라 다름)
  - 학과 전선 학점: 최소 12학점에서 15학점 이상 필요

이 정보는 2008년부터 2012년까지의 데이터를 기준으로 하며, 최신 정보는 학과의 내규에 따라 변경될 수 있습니다.


In [15]:
print(response2['output'])

The latest graduation requirements for the Department of Industrial Engineering at Seoul National University include completing core courses specific to the department, with options for interdisciplinary programs and double majors. Specific courses such as Production Management, Economic Engineering, Quality Management, and Linear and Nonlinear Optimization are required. Students must also meet a minimum credit requirement, which varies by the year of admission. For double majors, students must complete all core courses from the Industrial Engineering department, excluding certain courses like 'Human Engineering Experiment' and 'Understanding Industrial Engineering', and achieve a minimum of 39 credits. Additional credits from approved courses at foreign institutions may also be recognized up to a maximum of 9 credits.


In [16]:
print(response3['output'])

서울대학교 학업 이수 현황 페이지를 성공적으로 열었습니다. 해당 페이지에서 직접 자신의 학업 이수 현황을 확인할 수 있습니다.
